In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

dn = '/Users/aa/lab/kaggle/couponpurchase/data/'

## user_list.csv
22873 users

In [ ]:
users = pd.read_csv(dn+'common/user_list.csv', 
                    header=0, 
                    parse_dates=['REG_DATE', 'WITHDRAW_DATE'])

def add_col(df, cname, suffix, fn):
    df[cname+'_'+suffix] = df[cname].map(fn)

fn_yyyymm = lambda x: int('{year}{month:02d}'.format(year=x.year,month=x.month)) if x is not pd.NaT else -1
fn_dayofwk = lambda x: int('{day}'.format(day=x.dayofweek)) if x is not pd.NaT else -1
fn_hourofd = lambda x: int('{hour}'.format(hour=x.hour)) if x is not pd.NaT else -1

#add_col(users, 'REG_DATE', 'YYYYMM', fn_yyyymm)
#add_col(users, 'REG_DATE', 'DAYOFWK', fn_dayofwk)
#add_col(users, 'REG_DATE', 'HOUROFD', fn_hourofd)

#add_col(users, 'WITHDRAW_DATE', 'YYYYMM', fn_yyyymm)
#add_col(users, 'WITHDRAW_DATE', 'DAYOFWK', fn_dayofwk)
#add_col(users, 'WITHDRAW_DATE', 'HOUROFD', fn_hourofd)

users.head()

In [ ]:
users.info()

#### REG_DATE

In [ ]:
print('RegDt: Min={}, Max={}'.format(users['REG_DATE'].min(),
                                     users['REG_DATE'].max()))
print('WdDt : Min={}, Max={}'.format(users['WITHDRAW_DATE'].min(),
                                     users['WITHDRAW_DATE'].max()))

<u>How were registrations and withdrawals distributed over time</u>

In [ ]:
users['REG_DATE_YYYYMM'].value_counts().sort_index().plot(kind='bar')

In [ ]:
withdraw_not_minus = users.loc[users['WITHDRAW_DATE_YYYYMM']!=-1, ['WITHDRAW_DATE_YYYYMM']]
withdraw_not_minus['WITHDRAW_DATE_YYYYMM'].value_counts().sort_index().plot(kind='bar')

In [ ]:
def print_date_dist(title, se, fn):
    wise = se.map(fn)
    print('\n'+title)
    print(wise.value_counts().sort_index())

def print_yearmonwise(title, se):
    print_date_dist(title, se, 
                    lambda x: int('{year}{month:02d}'.format(year=x.year,month=x.month)) if x is not pd.NaT else -1)

def print_weekdaywise(title, se):
    print_date_dist(title, se, 
                    lambda x: int('{day}'.format(day=x.dayofweek)) if x is not pd.NaT else -1)

def print_hourwise(title, se):
    print_date_dist(title, se, 
                    lambda x: int('{hour}'.format(hour=x.hour)) if x is not pd.NaT else -1)

# print_yearmonwise('Registrations-ByYearMon', regdt)
# print_yearmonwise('Withdrawals-ByYearMon', wddt)
# print_weekdaywise('Registrations-ByWeekday', regdt)
# print_weekdaywise('Withdrawals-ByWeekday', wddt)
# print_hourwise('Registrations-ByHourofday', regdt)
# print_hourwise('Withdrawals-ByHourofday', wddt)

<u>How were registrations/withdrawals distributed by day of week</u>  

Most registrations around Mon/Tue/Wed. Most withdrawals around Wed/Thu, least on Fri

In [ ]:
# Monday=0, Sunday=6
fig, axes = plt.subplots(1,2)
sns.countplot(x='REG_DATE_DAYOFWK', data=users, order=range(0,7,1), ax=axes[0])
sns.countplot(x='WITHDRAW_DATE_DAYOFWK', data=users, order=range(0,7,1), ax=axes[1])

<u>How ware registrations/withdrawals distributed by hour of day</u>

In [ ]:
fig, axes = plt.subplots(1,2)
sns.countplot(x='REG_DATE_HOUROFD', data=users, order=range(0,24,1), ax=axes[0])
sns.countplot(x='WITHDRAW_DATE_HOUROFD', data=users, order=range(0,24,1), ax=axes[1])

#### SEX_ID
52% males, 48% females. All users have SEX_ID entered.

In [ ]:
print(users['SEX_ID'].value_counts())
print(len(users))

#### AGE
Most customers around 30-55yrs old

In [ ]:
print('Age: Min={}, Max={}'.format(users['AGE'].min(),
                                   users['AGE'].max()))
users['AGE'].hist(bins=30)

#### PREF_NAME
All 47 prefectures in Japan can be found in the dataset

In [ ]:
print(users['PREF_NAME'].describe())
print('All records (with/without PREF_NAME)', len(users['PREF_NAME']))
sns.countplot(x='PREF_NAME', data=users)

## coupon_list_train.csv

19413 coupons in training set

In [2]:
cpn_train = pd.read_csv(dn+'train/coupon_list_train.csv', 
                    header=0, 
                    parse_dates=['DISPFROM', 'DISPEND', 'VALIDFROM', 'VALIDEND'])
cpn_train.head(2)

,CAPSULE_TEXT,GENRE_NAME,PRICE_RATE,CATALOG_PRICE,DISCOUNT_PRICE,DISPFROM,DISPEND,DISPPERIOD,VALIDFROM,VALIDEND,...,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY,large_area_name,ken_name,small_area_name,COUPON_ID_hash
0,グルメ,グルメ,50,3000,1500,2011-07-08 12:00:00,2011-07-09 12:00:00,1,2011-07-10,2011-12-08,...,1,0,0,1,1,0,関東,埼玉県,埼玉,6b263844241eea98c5a97f1335ea82af
1,グルメ,グルメ,51,2080,1000,2011-07-01 12:00:00,2011-07-02 12:00:00,1,2011-07-03,2011-12-04,...,1,1,1,1,1,1,関東,千葉県,千葉,cc031f250e8bad1e24060263b9fc0ddd


In [4]:
cpn_train.describe()

,PRICE_RATE,CATALOG_PRICE,DISCOUNT_PRICE,DISPPERIOD,VALIDPERIOD,USABLE_DATE_MON,USABLE_DATE_TUE,USABLE_DATE_WED,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY
count,19413.000000,19413.000000,19413.000000,19413.000000,13266.000000,12408.000000,12408.000000,12408.000000,12408.000000,12408.000000,12408.000000,12408.000000,12408.000000,12408.000000
mean,58.478391,11818.368258,4332.877659,3.166950,125.955902,0.936009,0.953901,0.982189,0.981867,1.000242,1.067618,0.921422,0.938024,1.044246
std,11.266571,16881.898880,5459.667448,1.346859,46.599249,0.309854,0.314045,0.217525,0.198105,0.319562,0.563260,0.424181,0.399619,0.534511
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,50.000000,3675.000000,1550.000000,2.000000,89.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,53.000000,6500.000000,2750.000000,3.000000,128.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,65.000000,13650.000000,4800.000000,4.000000,177.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,100.000000,680000.000000,100000.000000,36.000000,179.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000


In [ ]:
cpn_train.USABLE_DATE_BEFORE_HOLIDAY.value_counts()

In [6]:
cpn_train[cpn_train.COUPON_ID_hash=='6d0a997acf143496aa621867244efac7']

,CAPSULE_TEXT,GENRE_NAME,PRICE_RATE,CATALOG_PRICE,DISCOUNT_PRICE,DISPFROM,DISPEND,DISPPERIOD,VALIDFROM,VALIDEND,...,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY,large_area_name,ken_name,small_area_name,COUPON_ID_hash
18874,その他,その他のクーポン,90,1000,100,2012-06-19 12:00:00,2012-06-23 12:00:00,4,2012-06-26,2012-07-31,...,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺,6d0a997acf143496aa621867244efac7


## coupon_list_test.csv
310 coupons in test set

In [ ]:
cpn_test = pd.read_csv(dn+'test/coupon_list_test.csv', 
                    header=0, 
                    parse_dates=['DISPFROM', 'DISPEND', 'VALIDFROM', 'VALIDEND'])
cpn_test.info()

## coupon_visit_train.csv
Browsing history of users during training period. 2.8 Mil rows

Views per month

In [ ]:
fn = lambda x: int('{year}{month:02d}'.format(year=x.year,month=x.month)) if x is not pd.NaT else -1
views.groupby(views.I_DATE.map(fn)).count()

Distinct users viewing per month

In [ ]:
fn = lambda x: int('{year}{month:02d}'.format(year=x.year,month=x.month)) if x is not pd.NaT else -1
views.groupby(views.I_DATE.map(fn)).USER_ID_hash.nunique()

## coupon_detail_train.csv

Purchase log during training-set time period (not provided for test-set period).  
168996 purchases

In [2]:
prch = pd.read_csv(dn+'train/coupon_detail_train.csv', 
                    header=0, 
                    parse_dates=['I_DATE'])
prch.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168996 entries, 0 to 168995
Data columns (total 6 columns):
ITEM_COUNT         168996 non-null int64
I_DATE             168996 non-null datetime64[ns]
SMALL_AREA_NAME    168996 non-null object
PURCHASEID_hash    168996 non-null object
USER_ID_hash       168996 non-null object
COUPON_ID_hash     168996 non-null object
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 9.0+ MB


In [ ]:
prch.head()

In [ ]:
for i, p in enumerate(prch.index):
    area_sa = area_train[area_train.COUPON_ID_hash==prch.iloc[p].COUPON_ID_hash].SMALL_AREA_NAME.values
    prch_sa = prch.iloc[p].SMALL_AREA_NAME
    cond = prch_sa in area_sa.tolist()
    if not cond:
        print('\n=============================')
        print(str(i+1)+'. Coupon Id:', prch.iloc[p].COUPON_ID_hash)
        print('Bought by:', prch.iloc[p].USER_ID_hash)
        print('---- Purchases:SmallAreaName ----')
        print(prch.iloc[p].SMALL_AREA_NAME)
        print('---- Users:PrefName ----')
        print(users[users.USER_ID_hash==prch.iloc[p].USER_ID_hash].PREF_NAME.values)
        print('---- Coupons:MerchantSmallArea ----')
        print(cpn_train[cpn_train.COUPON_ID_hash==prch.iloc[p].COUPON_ID_hash].small_area_name.values)
        print('---- Areas:ListingSmallArea ----')
        print(area_train[area_train.COUPON_ID_hash==prch.iloc[p].COUPON_ID_hash].SMALL_AREA_NAME.values)

In [ ]:
tmp = prch.groupby('USER_ID_hash').count()['PURCHASEID_hash']
tmp = tmp.copy()
tmp.sort(ascending=False)
tmp.hist(bins=20)

In [3]:
prch.groupby('USER_ID_hash').count()

,ITEM_COUNT,I_DATE,SMALL_AREA_NAME,PURCHASEID_hash,COUPON_ID_hash
USER_ID_hash,,,,,
0000b53e182165208887ba65c079fc21,1,1,1,1,1
00035b86e6884589ec8d28fbf2fe7757,1,1,1,1,1
0005b1068d5f2b8f2a7c978fcfe1ca06,2,2,2,2,2
000cc06982785a19e2a2fdb40b1c9d59,21,21,21,21,21
0013518e41c416cd6a181d277dd8ca0b,4,4,4,4,4
001acdee812a18acfd7509172bed5700,13,13,13,13,13
001fd7876e3aa29393537c6baf308e43,1,1,1,1,1
002383753c1e5d6305c8aff6f89e26d6,1,1,1,1,1
0025cae7997d25ea5cf8851bb099c798,9,9,9,9,9


In [ ]:
len(prch.USER_ID_hash.unique())

YearMonth-wise purchases

In [ ]:
fn = lambda x: int('{year}{month:02d}'.format(year=x.year,month=x.month)) if x is not pd.NaT else -1
wise = prch.I_DATE.map(fn)
wise.value_counts().sort_index().plot(kind='bar')

Number of users purchasing per month

In [ ]:
fn = lambda x: int('{year}{month:02d}'.format(year=x.year,month=x.month)) if x is not pd.NaT else -1
prch.groupby(purchases.I_DATE.map(fn)).USER_ID_hash.nunique()

## coupon_area_train.csv

coupon listing area for the training set coupons.  
138185 rows

In [4]:
area_train = pd.read_csv(dn+'train/coupon_area_train.csv', 
                    header=0)
print(area_train.info())
area_train.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 138185 entries, 0 to 138184
Data columns (total 3 columns):
SMALL_AREA_NAME    138185 non-null object
PREF_NAME          138185 non-null object
COUPON_ID_hash     138185 non-null object
dtypes: object(3)
memory usage: 4.2+ MB
None


,SMALL_AREA_NAME,PREF_NAME,COUPON_ID_hash
0,埼玉,埼玉県,6b263844241eea98c5a97f1335ea82af
1,千葉,千葉県,cc031f250e8bad1e24060263b9fc0ddd
2,千葉,千葉県,ba5e9b7453ca52ff711635a5d2e8102d
3,千葉,千葉県,3e1ffbedca3569f9e8032d401e8cb4e6
4,千葉,千葉県,782934b6c815b4030ea204eef7d4a734


In [ ]:
len(area_train.COUPON_ID_hash.unique())

## coupon_area_test.csv

coupon listing area for the test set coupons.  
2165 rows

In [ ]:
area_test = pd.read_csv(dn+'test/coupon_area_test.csv', 
                    header=0)
print(area_test.info())
area_test.head()

In [ ]:
len(area_test.COUPON_ID_hash.unique())

## prefecture_locations.csv

In [ ]:
prfctr_loc = pd.read_csv(dn+'common/prefecture_locations.csv', 
                         header=0, encoding='utf-8-sig')
print(prfctr_loc.info())
prfctr_loc.head()

In [ ]:
print(prfctr_loc.LATITUDE.describe())
print(prfctr_loc.LONGITUDE.describe())

## sample_submission.csv

In [ ]:
sample_subm = pd.read_csv(dn+'misc/sample_submission.csv', 
                    header=0)
print(sample_subm.info())
sample_subm.head()